# Fundamentals of Data Analysis Assignment
## Autumn 2021

---
<br>

## Part 2 - CAO points

![CAO_logo](./Images/cao.png)
---

<br>

# A detailed comparison of CAO points in 2019, 2020, and 2021

The brief was to analyse the CAO points for the years 2019, 2020, and 2021. The main tasks were to download the data from the CAO website, analyse the data using pandas and also to use visualisations to help better explain this analysis.

<br>

### Downloading the data

The data can be found at the following links which we save as variables to be used later.

In [1]:
url2021 = 'http://www2.cao.ie/points/l8.php'
url2020 = 'http://www2.cao.ie/points/CAOPointsCharts2020.xlsx'
url2019 = 'http://www2.cao.ie/points/lvl8_19.pdf'

A quick glance at the file extensions tells us we are dealing with three different file types so scraping the data is going to involve a few different methods.
To begin we import all the necessary libraries for the analysis and visualisations. These are shortened as per convention and economy of space.

In [2]:
# basic data analysis
import pandas as pd

# numerical arrays
import numpy as np

# plotting
import matplotlib.pyplot as plt

# Dates and times
import datetime as dt

# Regular expressions
import re

# HTTP requests
import requests as rq

# for downloading and saving excel file
import urllib.request as urlrq

# working with csv files
import csv

# for pdf files
import tabula

We're going to use a timestamp to name the different updates of each downloaded and saved file. We use the `datetime` library for the current date and time which is then converted into a string using the `strftime` method. `%Y%m%d_%H%M%S` specifies the format we want the date/time in.

In [3]:
# get the current date and time
now = dt.datetime.now()

# format as a string
nowstr = now.strftime('%Y%m%d_%H%M%S')

<br>

## 2021 Points

The url - http://www2.cao.ie/points/l8.php - returns a plain text file. For analysis we need to download that, extract only the information we need, and then convert it to a csv file. The first part of code below uses the `requests` library to fetch the data. A path is created using this stringified datetime preceded by the folder data and CAO2021. This timestamp could serve a dual purpose as the data from 2021 is still being updated (at time of writing).

In [4]:
# fetch the cao url
resp = rq.get(url2021)

# Create a filepath for the original data using the datetime
path2021 = 'data/CAO2021_' + nowstr + '.html'

# confirm it's working (if we get a '200' response message)
resp

<Response [200]>

We then save the original file which the following code. 

In [5]:
# save the original html file
with open(path2021, 'w') as f:
    f.write(resp.text)

During the lectures it was discovered that some characters weren't being displayed properly. The issues lay with the fadas in the Irish language words, plus a stray 'em' hyphen. The server stated that the page should be decoded with **iso-8859-1** but this didn't allow for these particular characters. We changed the decoding to **cp1252** which solved the issue.

In [6]:
# the server uses the wrong encoding, fix it
original_encoding = resp.encoding

# change to cp1252
resp.encoding = 'cp1252'

<br>

#### Using regular expressions to isolate the data we want

On inspection the file contains a lot of information we don't need for the analysis such as headings, links, college names, etc., so the next challenge was to isolate only what was needed, i.e. 1) course code 2) course name 3) points.
The following code uses a **regular expression** to identify only the lines that match the expression.
- `([A-Z]{2}[0-9]{3})` = represents course code - i.e. CW078
- followed by 2 spaces
- `(.*)` represents an amount of text. The dot `.` serves as a wildcard and the `*` means zero or more of.

In [7]:
# compile the regular expression for matching lines
re_course = re.compile(r'([A-Z]{2}[0-9]{3})(.*)')

A function created in the lecture to isolate # and *. **need to work out how to use it!**

In [8]:
# function to isolate # and * 
# def points_to_array(s):
#     portfolio = ''
#     if s[0] == '#':
#         portfolio = '#'
#     random = ''
#     if s[-1] == '*':
#         portfolio = '*'
#     points = ''
#     for i in s:
#         if i.isdigit():
#             points = points + i
#     return [points, portfolio, random]

We create a new path for saving the extracted data as a csv file.

In [9]:
# path for csv file
path2021b = 'data/CAO2021_csv_' + nowstr + '.csv'

We then loop through these lines and save then to a csv file to be also stored in the **data** folder. 

In [10]:
# keep track of courses
no_lines = 0

with open(path2021b, 'w') as f:
    # write a header row
    f.write(','.join(["Course_Code", "Course_Title", "PointsR1_2021", "PointsR2_2021"]) + '\n')
    # loop through lines of response
    for line in resp.iter_lines():
        # decode to cp1252
        dline = line.decode('cp1252')
        # match only the lines we want - ones representing courses
        if re_course.fullmatch(dline):
            # add to line counter
            no_lines = no_lines + 1
            # course code (first 5 characters)
            course_code = dline[:5]
            # course_title
            course_title = dline[7:57].strip()
            # round 1 points
            course_points = re.split(' +', dline[60:])
            if len(course_points) != 2:
                course_points = course_points[:2]
            # join the fields using a comma
            # split the line on 2 spaces or more
            linesplit = (course_code, course_title, course_points[0], course_points[1])
            f.write(','.join(linesplit) + '\n')

Just to confirm we have every line we need, we print out the total number of lines and then try to verify that online. 

In [11]:
# prints the total number of lines    
print(f"Total number of lines is {no_lines}.")

Total number of lines is 949.


**N.B.** It was verified on 8/11 against the original data and we're good!

We open the new csv file with the `pandas` package to inspect.

In [12]:
# open the csv file and save to variable df2021
df2021 = pd.read_csv(path2021b, encoding='cp1252') 

# have a look at the first 10 rows
df2021.head(10)

,Course_Code,Course_Title,PointsR1_2021,PointsR2_2021
0,AL801,Software Design for Virtual Reality and Gaming,300,NaN
1,AL802,Software Design in Artificial Intelligence for...,313,NaN
2,AL803,Software Design for Mobile Apps and Connected ...,350,NaN
3,AL805,Computer Engineering for Network Infrastructure,321,NaN
4,AL810,Quantity Surveying,328,NaN
5,AL811,Civil Engineering,NaN,NaN
6,AL820,Mechanical and Polymer Engineering,327,NaN
7,AL830,General Nursing,451*,444
8,AL832,Mental Health Nursing,440*,431
9,AL835,Pharmacology,356,NaN


In [13]:
df2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 949 entries, 0 to 948
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Course_Code    949 non-null    object
 1   Course_Title   949 non-null    object
 2   PointsR1_2021  928 non-null    object
 3   PointsR2_2021  258 non-null    object
dtypes: object(4)
memory usage: 29.8+ KB


Looks like all of the points are of datatype object. We need to change that to numeric values. Also we have some non-numeric characters so we'll have to get rid of them first and just leave the numbers. I found a solution here - https://pretagteam.com/question/remove-characters-from-pandas-column. By looking through the dataset I've identified all of the things I want to remove and replace them with blank space.

In [14]:
df2021 = df2021.replace('#', '', regex = True)
df2021 = df2021.replace('AQA', '', regex = True)
df2021 = df2021.replace('\*', '', regex = True)
df2021 = df2021.replace('\+matric', '', regex = True)
df2021 = df2021.replace('\+mat', '', regex = True)
df2021 = df2021.replace('e\)', '', regex = True)
df2021

,Course_Code,Course_Title,PointsR1_2021,PointsR2_2021
0,AL801,Software Design for Virtual Reality and Gaming,300,NaN
1,AL802,Software Design in Artificial Intelligence for...,313,NaN
2,AL803,Software Design for Mobile Apps and Connected ...,350,NaN
3,AL805,Computer Engineering for Network Infrastructure,321,NaN
4,AL810,Quantity Surveying,328,NaN
...,...,...,...,...
944,WD211,Creative Computing,270,NaN
945,WD212,Recreation and Sport Management,262,NaN
946,WD230,Mechanical and Manufacturing Engineering,230,230
947,WD231,Early Childhood Care and Education,266,NaN


In [15]:
df2021['PointsR1_2021'] = pd.to_numeric(df2021['PointsR1_2021'])
df2021['PointsR2_2021'] = pd.to_numeric(df2021['PointsR2_2021'])
df2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 949 entries, 0 to 948
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Course_Code    949 non-null    object 
 1   Course_Title   949 non-null    object 
 2   PointsR1_2021  923 non-null    float64
 3   PointsR2_2021  255 non-null    float64
dtypes: float64(2), object(2)
memory usage: 29.8+ KB


In [16]:
df2021.describe()

,PointsR1_2021,PointsR2_2021
count,923.000000,255.000000
mean,407.666306,414.749020
std,128.706224,141.693386
min,57.000000,60.000000
25%,303.000000,293.500000
50%,391.000000,424.000000
75%,499.000000,521.500000
max,1028.000000,904.000000


We need to combine the 2 points and take the lower of each.

In [17]:
# # df1 = pd.DataFrame({'A': [5, 0], 'B': [2, 4]})
# # df2 = pd.DataFrame({'A': [1, 1], 'B': [3, 3]})
# # df1.combine(df2, np.minimum)

# df1 = pd.DataFrame({'A': [0, 0], 'B': [4, 4]})
# df2 = pd.DataFrame({'A': [1, 1], 'B': [3, 3]})
# take_smaller = lambda s1, s2: s1 if s1.sum() < s2.sum() else s2
# df1.combine(df2, take_smaller)

# df1 = df2021['PointsR1_2021']
# df2 = df2021['PointsR2_2021']
# finalPoints2021 = df1.combine(df2, np.minimum, overwrite=False)

df1 = df2021['PointsR1_2021']
df2 = df2021['PointsR2_2021']
take_smaller = lambda s1, s2: s1 if s1 < s2 else s2
finalPoints2021 = df2.combine(df1, take_smaller)
finalPoints2021

0      300.0
1      313.0
2      350.0
3      321.0
4      328.0
       ...  
944    270.0
945    262.0
946    230.0
947    266.0
948    261.0
Length: 949, dtype: float64

In [18]:
df2021['finalPoints2021'] = finalPoints2021
df2021.head(20)

,Course_Code,Course_Title,PointsR1_2021,PointsR2_2021,finalPoints2021
0,AL801,Software Design for Virtual Reality and Gaming,300.0,NaN,300.0
1,AL802,Software Design in Artificial Intelligence for...,313.0,NaN,313.0
2,AL803,Software Design for Mobile Apps and Connected ...,350.0,NaN,350.0
3,AL805,Computer Engineering for Network Infrastructure,321.0,NaN,321.0
4,AL810,Quantity Surveying,328.0,NaN,328.0
5,AL811,Civil Engineering,NaN,NaN,NaN
6,AL820,Mechanical and Polymer Engineering,327.0,NaN,327.0
7,AL830,General Nursing,451.0,444.0,444.0
8,AL832,Mental Health Nursing,440.0,431.0,431.0
9,AL835,Pharmacology,356.0,NaN,356.0


## 2020 Points

We move on to the 2020 points.
The 2020 data is already in an excel spreadsheet so a different approach is needed!
We first save the original data and name it again using the timestamp created above.

In [19]:
# create a new path
path2020 = 'data/CAO2020_' + nowstr + '.html'

# Read and store content of excel file from a URL 
urlrq.urlretrieve(url2020, path2020)

('data/CAO2020_20211129_234108.html',
 <http.client.HTTPMessage at 0x7fc3609fb490>)

We read it in again using `pandas` skipping the first 10 rows which on inspection contained stuff we didn't need.

In [20]:
# skip the first 10 rows
df2020 = pd.read_excel (url2020, skiprows=10)

# have a look at the first ten rows
df2020.head()

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We can see that we have a lot more data than the 2021 version plus different headers. So straight away I'd like to rename the matching headers so we can look at both files together.

In [21]:
# change the necessary headers - Course_Code,Course_Title,PointsR1,PointsR2
df2020 = df2020.rename(columns={'COURSE TITLE': 'Course_Title', 'COURSE CODE2': 'Course_Code', 
                                'R1 POINTS': 'PointsR1_2020', 'R2 POINTS': 'PointsR2_2020'})
df2020.head()

,CATEGORY (i.e.ISCED description),Course_Title,Course_Code,PointsR1_2020,R1 Random *,PointsR2_2020,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now to isolate the relevent rows and save them to a new csv file

In [22]:
# extract the relevent columns and reorder
df2020 = df2020[['Course_Code','Course_Title','PointsR1_2020','PointsR2_2020']]

# have a look
df2020.head()

,Course_Code,Course_Title,PointsR1_2020,PointsR2_2020
0,AC120,International Business,209,NaN
1,AC137,Liberal Arts,252,NaN
2,AD101,"First Year Art & Design (Common Entry,portfolio)",#+matric,NaN
3,AD102,Graphic Design and Moving Image Design (portfo...,#+matric,NaN
4,AD103,Textile & Surface Design and Jewellery & Objec...,#+matric,NaN


The new files look similar now but we can see straight away that 2020 has considerable more rows than 2021! Also we can see a few non-numeric values in the points column. We'll address both of these now.  
In the meantime we save this version to a new file again using the timestamp to name the file.

In [23]:
df2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1464 entries, 0 to 1463
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Course_Code    1464 non-null   object
 1   Course_Title   1464 non-null   object
 2   PointsR1_2020  1437 non-null   object
 3   PointsR2_2020  333 non-null    object
dtypes: object(4)
memory usage: 45.9+ KB


In [24]:
df2020 = df2020.replace('#', '', regex = True)
df2020 = df2020.replace('AQA', '', regex = True)
df2020 = df2020.replace('\*', '', regex = True)
df2020 = df2020.replace('\+matric', '', regex = True)
df2020 = df2020.replace('\+mat', '', regex = True)
df2020 = df2020.replace('e\)', '', regex = True)
df2020.head()

,Course_Code,Course_Title,PointsR1_2020,PointsR2_2020
0,AC120,International Business,209,NaN
1,AC137,Liberal Arts,252,NaN
2,AD101,"First Year Art & Design (Common Entry,portfolio)",,NaN
3,AD102,Graphic Design and Moving Image Design (portfo...,,NaN
4,AD103,Textile & Surface Design and Jewellery & Objec...,,NaN


Again we need to change the points value to a. numeric datatype.

In [25]:
df2020['PointsR1_2020'] = pd.to_numeric(df2020['PointsR1_2020'])
df2020['PointsR2_2020'] = pd.to_numeric(df2020['PointsR2_2020'])
df2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1464 entries, 0 to 1463
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Course_Code    1464 non-null   object 
 1   Course_Title   1464 non-null   object 
 2   PointsR1_2020  1394 non-null   float64
 3   PointsR2_2020  316 non-null    float64
dtypes: float64(2), object(2)
memory usage: 45.9+ KB


In [26]:
df2020.describe()

,PointsR1_2020,PointsR2_2020
count,1394.000000,316.000000
mean,350.995696,334.329114
std,134.433752,142.016943
min,55.000000,100.000000
25%,252.250000,212.000000
50%,316.500000,305.000000
75%,433.000000,462.750000
max,1088.000000,768.000000


In [27]:
df1 = df2020['PointsR1_2020']
df2 = df2020['PointsR2_2020']
take_smaller = lambda s1, s2: s1 if s1 < s2 else s2
finalPoints2020 = df2.combine(df1, take_smaller)
finalPoints2020

# df1 = df2021['PointsR1_2021']
# df2 = df2021['PointsR2_2021']
# take_smaller = lambda s1, s2: s1 if s1 < s2 else s2
# finalPoints2021 = df2.combine(df1, take_smaller)
# finalPoints2021

0       209.0
1       252.0
2         NaN
3         NaN
4         NaN
        ...  
1459    188.0
1460    279.0
1461    271.0
1462    270.0
1463    253.0
Length: 1464, dtype: float64

In [28]:
df2020['finalPoints2020'] = finalPoints2020
df2020.head()

,Course_Code,Course_Title,PointsR1_2020,PointsR2_2020,finalPoints2020
0,AC120,International Business,209.0,NaN,209.0
1,AC137,Liberal Arts,252.0,NaN,252.0
2,AD101,"First Year Art & Design (Common Entry,portfolio)",NaN,NaN,NaN
3,AD102,Graphic Design and Moving Image Design (portfo...,NaN,NaN,NaN
4,AD103,Textile & Surface Design and Jewellery & Objec...,NaN,NaN,NaN


In [29]:
# saving updated pandas dataframe as csv file
# creating path
path2020b = 'data/CAO2020_' + nowstr + '.csv'
# writing to csv
df2020.to_csv(path2020b)

<br>

## 2019 Points

These are in PDF format! A bit of googling found the `tabula` library which seems to deal with PDFs very easily.  
Link to tabula - https://github.com/chezou/tabula-py

In [30]:
# access the pdf file using parameters in referenced link above
df = tabula.read_pdf(url2019, stream=True, pages="all")

Seems to work!  
Next to convert it to a csv file and name it using the timestamp method again, and save it to the data folder

In [31]:
# convert PDF into CSV file
tabula.convert_into(url2019, 'data/CAO2019_' + nowstr + '.csv', output_format="csv", pages='all')

We read it in again with `pandas`.

In [32]:
# reads in the csv file
df2019 = pd.read_csv("data/CAO2019_20211101_122559.csv")

# have a look
df2019.head(10)

,Course Code,INSTITUTION and COURSE,EOS,Mid
0,NaN,Athlone Institute of Technology,NaN,NaN
1,AL801,Software Design with Virtual Reality and Gaming,304,328
2,AL802,Software Design with Cloud Computing,301,306
3,AL803,Software Design with Mobile Apps and Connected...,309,337
4,AL805,Network Management and Cloud Infrastructure,329,442
5,AL810,Quantity Surveying,307,349
6,AL820,Mechanical and Polymer Engineering,300,358
7,AL830,General Nursing,410,429
8,AL832,Psychiatric Nursing,387,403
9,AL836,Nutrition and Health Science,352,383


We have an issue with the univeristy columns in that they are taking up rows! We need to be able to delete those rows and a bit of googling found the following solution - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html

In [33]:
# deletes rows where there are blanks
df2019 = df2019.dropna()

# have a look
df2019.head(10)

,Course Code,INSTITUTION and COURSE,EOS,Mid
1,AL801,Software Design with Virtual Reality and Gaming,304,328
2,AL802,Software Design with Cloud Computing,301,306
3,AL803,Software Design with Mobile Apps and Connected...,309,337
4,AL805,Network Management and Cloud Infrastructure,329,442
5,AL810,Quantity Surveying,307,349
6,AL820,Mechanical and Polymer Engineering,300,358
7,AL830,General Nursing,410,429
8,AL832,Psychiatric Nursing,387,403
9,AL836,Nutrition and Health Science,352,383
10,AL837,Sports Science with Exercise Physiology,351,392


We also need to rename the headers to match the other years files, but we have an issue - we don't have the round 1 and round 2 points! According to https://www.independent.ie/life/family/learning/understanding-your-cao-course-guide-26505318.htmlINSTITUTION EOS is the 
>*".. FINAL CUT-OFF points, in other words, the points score achieved by the last applicant being offered a place on that course in 2008. The second column gives the MID figure, that is, the points score of the applicant midway between the highest and the lowest applicant being offered a place."*

EOS roughly compares to Round 2 offers maybe? We have an EOS column and an EOS Mid-point column for 2020 but not 2021. We'll address that later!

In [34]:
# rename columns for Course_code and Course_title but leave the others
df2019 = df2019.rename(columns={'INSTITUTION and COURSE': 'Course_Title', 
                                'Course Code': 'Course_Code', 'EOS': 'EOS_2019'})
df2019.head()

,Course_Code,Course_Title,EOS_2019,Mid
1,AL801,Software Design with Virtual Reality and Gaming,304,328
2,AL802,Software Design with Cloud Computing,301,306
3,AL803,Software Design with Mobile Apps and Connected...,309,337
4,AL805,Network Management and Cloud Infrastructure,329,442
5,AL810,Quantity Surveying,307,349


In [35]:
df2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 915 entries, 1 to 964
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Course_Code   915 non-null    object
 1   Course_Title  915 non-null    object
 2   EOS_2019      915 non-null    object
 3   Mid           915 non-null    object
dtypes: object(4)
memory usage: 35.7+ KB


In [68]:
# https://stackoverflow.com/questions/39173813/pandas-convert-dtype-object-to-int
df2019['EOS_2019'] = pd.to_numeric(df2019['EOS_2019'], errors='coerce')
finalPoints2019 = df2019['EOS_2019']
finalPoints2019

1      304.0
2      301.0
3      309.0
4      329.0
5      307.0
       ...  
960    221.0
961    271.0
962    275.0
963    274.0
964    273.0
Name: EOS_2019, Length: 915, dtype: float64

In [37]:
df2019.describe()

,EOS_2019
count,791.000000
mean,351.364096
std,88.284545
min,168.000000
25%,285.500000
50%,330.000000
75%,410.000000
max,601.000000


In [38]:
df2019['finalPoints2019'] = finalPoints2019
df2019.head()

,Course_Code,Course_Title,EOS_2019,Mid,finalPoints2019
1,AL801,Software Design with Virtual Reality and Gaming,304.0,328,304.0
2,AL802,Software Design with Cloud Computing,301.0,306,301.0
3,AL803,Software Design with Mobile Apps and Connected...,309.0,337,309.0
4,AL805,Network Management and Cloud Infrastructure,329.0,442,329.0
5,AL810,Quantity Surveying,307.0,349,307.0


We save the amended file using the same method as above.

In [39]:
# create a new path
path2019b = 'data/CAO2019_' + nowstr + '.csv'

# saving amended file to folder
df2019.to_csv(path2019b)

<br>

### Concat and join

Now we have the data in a useable form we start to do some work on it and prepare it for analysis.

We begin by isolating the course code and course title from each year and creating new variables for each.

In [40]:
# create new variable
courses2021 = df2021[['Course_Code', 'Course_Title']]

# print out first 5 rows
courses2021.head()

,Course_Code,Course_Title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying


In [41]:
# create new variable
courses2020 = df2020[['Course_Code', 'Course_Title']]

# print out first 5 rows
courses2020.head()

,Course_Code,Course_Title
0,AC120,International Business
1,AC137,Liberal Arts
2,AD101,"First Year Art & Design (Common Entry,portfolio)"
3,AD102,Graphic Design and Moving Image Design (portfo...
4,AD103,Textile & Surface Design and Jewellery & Objec...


In [42]:
# create new variable
courses2019 = df2019[['Course_Code', 'Course_Title']]

# print out first 5 rows
courses2019.head()

,Course_Code,Course_Title
1,AL801,Software Design with Virtual Reality and Gaming
2,AL802,Software Design with Cloud Computing
3,AL803,Software Design with Mobile Apps and Connected...
4,AL805,Network Management and Cloud Infrastructure
5,AL810,Quantity Surveying


We use `pandas.concat` to concatenate each of these variables to the one dataframe which we call `allCourses`.

In [43]:
allCourses = pd.concat([courses2021, courses2020, courses2019], ignore_index=True)
allCourses.head()

,Course_Code,Course_Title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying


We sort the values into alphbetical order based on course code.

In [44]:
# sort values
allCourses = allCourses.sort_values('Course_Code')

We show all duplicated rows using the `pandas.duplicated` method.

In [45]:
# drop duplicated rows
allCourses[allCourses.duplicated(keep=False)]

,Course_Code,Course_Title
175,AC120,International Business
949,AC120,International Business
2579,AC120,International Business
950,AC137,Liberal Arts
2580,AC137,Liberal Arts
...,...,...
3326,WD212,Recreation and Sport Management
945,WD212,Recreation and Sport Management
2412,WD230,Mechanical and Manufacturing Engineering
946,WD230,Mechanical and Manufacturing Engineering


We drop all the duplicated rows.

In [46]:
# copy of dataframe with duplicates removed
allCourses.drop_duplicates()

,Course_Code,Course_Title
175,AC120,International Business
950,AC137,Liberal Arts
2820,AD101,First Year Art & Design (Common Entry)
951,AD101,"First Year Art & Design (Common Entry,portfolio)"
311,AD101,First Year Art and Design (Common Entry portfo...
...,...,...
2410,WD211,Creative Computing
2411,WD212,Recreation and Sport Management
2412,WD230,Mechanical and Manufacturing Engineering
947,WD231,Early Childhood Care and Education


In [47]:
allCourses[allCourses.duplicated(subset=['Course_Code'])]

,Course_Code,Course_Title
949,AC120,International Business
2579,AC120,International Business
2580,AC137,Liberal Arts
176,AC137,Liberal Arts
951,AD101,"First Year Art & Design (Common Entry,portfolio)"
...,...,...
3325,WD211,Creative Computing
3326,WD212,Recreation and Sport Management
945,WD212,Recreation and Sport Management
946,WD230,Mechanical and Manufacturing Engineering


In [48]:
# drop duplicates based on code
# inplace changes original dataframe - default is opposite
allCourses.drop_duplicates(subset=['Course_Code'], inplace=True, ignore_index=True)
allCourses

,Course_Code,Course_Title
0,AC120,International Business
1,AC137,Liberal Arts
2,AD101,First Year Art & Design (Common Entry)
3,AD102,Graphic Design and Moving Image Design (portfo...
4,AD103,Textile & Surface Design and Jewellery & Objec...
...,...,...
1644,WD211,Creative Computing
1645,WD212,Recreation and Sport Management
1646,WD230,Mechanical and Manufacturing Engineering
1647,WD231,Early Childhood Care and Education


### Join to the points

In [63]:
# df2021.set_index('Course_Code', inplace=True)
# df2021.columns = ['Course_Title', 'PointsR1_2021', 'PointsR2_2021', 'finalPoints2021']

We set the index to 'Course_code'.

In [53]:
# set index
allCourses.set_index('Course_Code', inplace=True)

In [54]:
allCourses = allCourses.join(df2021[['finalPoints2021']])
allCourses.head()

,Course_Title,finalPoints2021
Course_Code,,
AC120,International Business,294.0
AC137,Liberal Arts,270.0
AD101,First Year Art & Design (Common Entry),554.0
AD102,Graphic Design and Moving Image Design (portfo...,538.0
AD103,Textile & Surface Design and Jewellery & Objec...,505.0


In [55]:
df2020_r1 = df2020[['Course_Code', 'finalPoints2020']]
df2020_r1

,Course_Code,finalPoints2020
0,AC120,209.0
1,AC137,252.0
2,AD101,NaN
3,AD102,NaN
4,AD103,NaN
...,...,...
1459,WD208,188.0
1460,WD210,279.0
1461,WD211,271.0
1462,WD212,270.0


In [56]:
df2020_r1.set_index('Course_Code', inplace=True)
df2020_r1

,finalPoints2020
Course_Code,
AC120,209.0
AC137,252.0
AD101,NaN
AD102,NaN
AD103,NaN
...,...
WD208,188.0
WD210,279.0
WD211,271.0


In [57]:
allCourses = allCourses.join(df2020_r1)
allCourses

,Course_Title,finalPoints2021,finalPoints2020
Course_Code,,,
AC120,International Business,294.0,209.0
AC137,Liberal Arts,270.0,252.0
AD101,First Year Art & Design (Common Entry),554.0,NaN
AD102,Graphic Design and Moving Image Design (portfo...,538.0,NaN
AD103,Textile & Surface Design and Jewellery & Objec...,505.0,NaN
...,...,...,...
WD211,Creative Computing,270.0,271.0
WD212,Recreation and Sport Management,262.0,270.0
WD230,Mechanical and Manufacturing Engineering,230.0,253.0


In [58]:
df2019_EOS = df2019[['Course_Code', 'finalPoints2019']]
df2019_EOS

,Course_Code,finalPoints2019
1,AL801,304.0
2,AL802,301.0
3,AL803,309.0
4,AL805,329.0
5,AL810,307.0
...,...,...
960,WD200,221.0
961,WD210,271.0
962,WD211,275.0
963,WD212,274.0


In [59]:
df2019_EOS.set_index('Course_Code', inplace=True)
df2019_EOS

,finalPoints2019
Course_Code,
AL801,304.0
AL802,301.0
AL803,309.0
AL805,329.0
AL810,307.0
...,...
WD200,221.0
WD210,271.0
WD211,275.0


In [60]:
allCourses = allCourses.join(df2019_EOS)
allCourses

,Course_Title,finalPoints2021,finalPoints2020,finalPoints2019
Course_Code,,,,
AC120,International Business,294.0,209.0,234.0
AC137,Liberal Arts,270.0,252.0,252.0
AD101,First Year Art & Design (Common Entry),554.0,NaN,NaN
AD102,Graphic Design and Moving Image Design (portfo...,538.0,NaN,NaN
AD103,Textile & Surface Design and Jewellery & Objec...,505.0,NaN,NaN
...,...,...,...,...
WD211,Creative Computing,270.0,271.0,275.0
WD212,Recreation and Sport Management,262.0,270.0,274.0
WD230,Mechanical and Manufacturing Engineering,230.0,253.0,273.0


Reorder again

In [61]:
allCourses = allCourses.sort_values('Course_Code')

In [62]:
allCourses.head()

,Course_Title,finalPoints2021,finalPoints2020,finalPoints2019
Course_Code,,,,
AC120,International Business,294.0,209.0,234.0
AC137,Liberal Arts,270.0,252.0,252.0
AD101,First Year Art & Design (Common Entry),554.0,NaN,NaN
AD102,Graphic Design and Moving Image Design (portfo...,538.0,NaN,NaN
AD103,Textile & Surface Design and Jewellery & Objec...,505.0,NaN,NaN


We save this version as a csv to file using the same method as above.

In [64]:
# create a new path to save file to
pathAllCourses = 'data/CAOAllYears_' + nowstr + '.csv'

# saving amended file to folder
allCourses.to_csv(pathAllCourses)

<br>

### Some analysis

So I had to convert the columns to a numeric type to perform any kind of analysis. Probably a much easier way of doing this?  
I use the **describe()** function to do some exploring.

In [65]:
allCourses['finalPoints2021'].describe()

count     923.000000
mean      404.913326
std       128.748376
min        57.000000
25%       301.500000
50%       391.000000
75%       497.500000
max      1028.000000
Name: finalPoints2021, dtype: float64

In [66]:
allCourses['finalPoints2020'].describe()

count    1394.000000
mean      348.797704
std       134.004567
min        55.000000
25%       252.000000
50%       313.000000
75%       430.000000
max      1088.000000
Name: finalPoints2020, dtype: float64

In [67]:
allCourses['finalPoints2019'].describe()

count    791.000000
mean     351.364096
std       88.284545
min      168.000000
25%      285.500000
50%      330.000000
75%      410.000000
max      601.000000
Name: finalPoints2019, dtype: float64

In [74]:
print(allCourses['finalPoints2019'].mean())
print(allCourses['finalPoints2020'].mean())
print(allCourses['finalPoints2021'].mean())

351.36409608091026
348.7977044476327
404.9133261105092


<br>

## References
---
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html

https://www.geeksforgeeks.org/adding-new-column-to-existing-dataframe-in-pandas/

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.combine.html

# THE END